In [1]:
import pandas as pd
import numpy as np
import dateutil.easter as easter
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

In [2]:
train_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/train.csv')
test_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/test.csv')

# The dates are read as strings and must be converted
for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df.date)
    df.set_index('date', inplace=True, drop=False)
train_df

,id,date,country,store,product,num_sold
date,,,,,,
2010-01-01,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
2010-01-01,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2010-01-01,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
2010-01-01,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
2010-01-01,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
...,...,...,...,...,...,...
2016-12-31,230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0
2016-12-31,230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0
2016-12-31,230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0


There are 6 countries, 3 stores and 5 products. For all 90 combinations of these, we have the sales data for 2557 days. The 2557 days are all days of the seven years 2010, 2011, 2012, 2013, 2014, 2015, 2016. There are no missing values.

* Countries: 6
* Stores: 3
* Products: 5

6×3×5=90

In [3]:
print(train_df.groupby(['country', 'store', 'product']).date.count())

print("First day:", train_df.date.min(), "   Last day:", train_df.date.max())

print("Number of days in seven years:", 365 * 7 + 2) # seven years including two leap year

print(90 * 2557, train_df.shape, train_df.date.isna().sum())

country    store              product           
Canada     Discount Stickers  Holographic Goose     2557
                              Kaggle                2557
                              Kaggle Tiers          2557
                              Kerneler              2557
                              Kerneler Dark Mode    2557
                                                    ... 
Singapore  Stickers for Less  Holographic Goose     2557
                              Kaggle                2557
                              Kaggle Tiers          2557
                              Kerneler              2557
                              Kerneler Dark Mode    2557
Name: date, Length: 90, dtype: int64
First day: 2010-01-01 00:00:00    Last day: 2016-12-31 00:00:00
Number of days in seven years: 2557
230130 (230130, 6) 0


In [4]:
train_df.groupby(['country', 'store', 'product']).num_sold.agg(['min', 'max', 'mean']).head(45)

min     max         mean
country store                product                                        
Canada  Discount Stickers    Holographic Goose      NaN     NaN          NaN
                             Kaggle               508.0  1216.0   715.652718
                             Kaggle Tiers         412.0  1002.0   592.579194
                             Kerneler             200.0   534.0   323.602113
                             Kerneler Dark Mode   211.0   660.0   377.781776
        Premium Sticker Mart Holographic Goose    200.0   449.0   248.382177
                             Kaggle              1146.0  2749.0  1712.174032
                             Kaggle Tiers        1004.0  2484.0  1417.701995
                             Kerneler             470.0  1291.0   773.023465
                             Kerneler Dark Mode   527.0  1493.0   904.393430
        Stickers for Less    Holographic Goose    200.0   380.0   228.816653
                             Kaggle              1040.0  2432.0  1449.021510
                             Kaggle Tiers         812.0  2099.0  1198.617912
                             Kerneler             407.0  1058.0   654.836918
                             Kerneler Dark Mode   464.0  1291.0   763.250293
Finland Discount Stickers    Holographic Goose     65.0   195.0    98.001564
                             Kaggle               515.0  1204.0   703.688698
                             Kaggle Tiers         404.0  1085.0   581.462652
                             Kerneler             208.0   561.0   317.497849
                             Kerneler Dark Mode   216.0   661.0   372.158389
        Premium Sticker Mart Holographic Goose    156.0   491.0   235.260461
                             Kaggle              1175.0  3173.0  1684.023074
                             Kaggle Tiers         977.0  2349.0  1388.125147
                             Kerneler             484.0  1382.0   758.843176
                             Kerneler Dark Mode   549.0  1555.0   890.979664
        Stickers for Less    Holographic Goose    132.0   396.0   198.955808
                             Kaggle              1024.0  2355.0  1424.984357
                             Kaggle Tiers         783.0  2113.0  1175.055925
                             Kerneler             419.0  1072.0   641.964803
                             Kerneler Dark Mode   468.0  1272.0   752.648025
Italy   Discount Stickers    Holographic Goose     44.0   135.0    70.846304
                             Kaggle               368.0   809.0   509.840438
                             Kaggle Tiers         287.0   728.0   421.778256
                             Kerneler             145.0   389.0   230.076261
                             Kerneler Dark Mode   154.0   463.0   268.933516
        Premium Sticker Mart Holographic Goose    111.0   318.0   170.382871
                             Kaggle               869.0  1905.0  1219.964411
                             Kaggle Tiers         672.0  1907.0  1008.123582
                             Kerneler             350.0   930.0   550.736801
                             Kerneler Dark Mode   372.0  1139.0   644.371529
        Stickers for Less    Holographic Goose     95.0   263.0   144.018381
                             Kaggle               713.0  1632.0  1031.843958
                             Kaggle Tiers         572.0  1377.0   852.334767
                             Kerneler             296.0   766.0   465.649980
                             Kerneler Dark Mode   327.0   898.0   544.719984

In [5]:
train_df.groupby(['country', 'store', 'product']).num_sold.agg(['min', 'max', 'mean']).tail(45)

min     max         mean
country   store                product                                        
Kenya     Discount Stickers    Holographic Goose      NaN     NaN          NaN
                               Kaggle                10.0    30.0    17.160735
                               Kaggle Tiers           8.0    25.0    14.109894
                               Kerneler               5.0    14.0     7.578990
                               Kerneler Dark Mode     5.0    17.0     8.824335
          Premium Sticker Mart Holographic Goose      5.0    11.0     5.941915
                               Kaggle                25.0    73.0    41.732499
                               Kaggle Tiers          22.0    60.0    34.460696
                               Kerneler              10.0    36.0    18.615565
                               Kerneler Dark Mode    11.0    42.0    21.827141
          Stickers for Less    Holographic Goose      5.0    10.0     5.470392
                               Kaggle                21.0    66.0    35.286273
                               Kaggle Tiers          18.0    52.0    29.040282
                               Kerneler               8.0    29.0    15.646461
                               Kerneler Dark Mode    10.0    34.0    18.412984
Norway    Discount Stickers    Holographic Goose    105.0   352.0   186.725851
                               Kaggle               855.0  2488.0  1339.298005
                               Kaggle Tiers         684.0  2084.0  1105.479077
                               Kerneler             355.0  1164.0   603.490418
                               Kerneler Dark Mode   381.0  1298.0   707.368792
          Premium Sticker Mart Holographic Goose    266.0   908.0   447.119671
                               Kaggle              2062.0  5939.0  3197.896754
                               Kaggle Tiers        1648.0  4958.0  2642.728197
                               Kerneler             812.0  2496.0  1441.943684
                               Kerneler Dark Mode   884.0  3127.0  1689.502151
          Stickers for Less    Holographic Goose    218.0   749.0   378.892061
                               Kaggle              1611.0  4444.0  2705.404380
                               Kaggle Tiers        1377.0  4308.0  2235.983183
                               Kerneler             673.0  2302.0  1223.681658
                               Kerneler Dark Mode   749.0  2740.0  1432.892452
Singapore Discount Stickers    Holographic Goose     69.0   200.0   113.131795
                               Kaggle               548.0  1259.0   814.008995
                               Kaggle Tiers         451.0  1057.0   671.281971
                               Kerneler             211.0   590.0   366.472038
                               Kerneler Dark Mode   253.0   701.0   428.336723
          Premium Sticker Mart Holographic Goose    167.0   469.0   271.564333
                               Kaggle              1265.0  2907.0  1937.877591
                               Kaggle Tiers        1085.0  2585.0  1604.389128
                               Kerneler             525.0  1443.0   875.784122
                               Kerneler Dark Mode   579.0  1719.0  1024.878373
          Stickers for Less    Holographic Goose    147.0   437.0   229.463043
                               Kaggle              1068.0  2534.0  1643.064138
                               Kaggle Tiers         908.0  2219.0  1356.942120
                               Kerneler             420.0  1191.0   741.365663
                               Kerneler Dark Mode   486.0  1421.0   867.225655

In [6]:
test_df.date.min(), test_df.date.max()

(Timestamp('2017-01-01 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [7]:
kk = train_df.groupby(['country', 'store', 'product']).num_sold.mean().unstack(level='store')

kk['Discount:StickersForLess'] = kk['Discount Stickers'] / kk['Stickers for Less']
kk['Discount:Premium'] = kk['Discount Stickers'] / kk['Premium Sticker Mart']
kk['StickersForLess:Premium'] = kk['Stickers for Less'] / kk['Premium Sticker Mart']

kk

store                         Discount Stickers  Premium Sticker Mart  \
country   product                                                       
Canada    Holographic Goose                 NaN            248.382177   
          Kaggle                     715.652718           1712.174032   
          Kaggle Tiers               592.579194           1417.701995   
          Kerneler                   323.602113            773.023465   
          Kerneler Dark Mode         377.781776            904.393430   
Finland   Holographic Goose           98.001564            235.260461   
          Kaggle                     703.688698           1684.023074   
          Kaggle Tiers               581.462652           1388.125147   
          Kerneler                   317.497849            758.843176   
          Kerneler Dark Mode         372.158389            890.979664   
Italy     Holographic Goose           70.846304            170.382871   
          Kaggle                     509.840438           1219.964411   
          Kaggle Tiers               421.778256           1008.123582   
          Kerneler                   230.076261            550.736801   
          Kerneler Dark Mode         268.933516            644.371529   
Kenya     Holographic Goose                 NaN              5.941915   
          Kaggle                      17.160735             41.732499   
          Kaggle Tiers                14.109894             34.460696   
          Kerneler                     7.578990             18.615565   
          Kerneler Dark Mode           8.824335             21.827141   
Norway    Holographic Goose          186.725851            447.119671   
          Kaggle                    1339.298005           3197.896754   
          Kaggle Tiers              1105.479077           2642.728197   
          Kerneler                   603.490418           1441.943684   
          Kerneler Dark Mode         707.368792           1689.502151   
Singapore Holographic Goose          113.131795            271.564333   
          Kaggle                     814.008995           1937.877591   
          Kaggle Tiers               671.281971           1604.389128   
          Kerneler                   366.472038            875.784122   
          Kerneler Dark Mode         428.336723           1024.878373   

store                         Stickers for Less  Discount:StickersForLess  \
country   product                                                           
Canada    Holographic Goose          228.816653                       NaN   
          Kaggle                    1449.021510                  0.493887   
          Kaggle Tiers              1198.617912                  0.494385   
          Kerneler                   654.836918                  0.494172   
          Kerneler Dark Mode         763.250293                  0.494964   
Finland   Holographic Goose          198.955808                  0.492580   
          Kaggle                    1424.984357                  0.493822   
          Kaggle Tiers              1175.055925                  0.494838   
          Kerneler                   641.964803                  0.494572   
          Kerneler Dark Mode         752.648025                  0.494465   
Italy     Holographic Goose          144.018381                  0.491925   
          Kaggle                    1031.843958                  0.494106   
          Kaggle Tiers               852.334767                  0.494850   
          Kerneler                   465.649980                  0.494097   
          Kerneler Dark Mode         544.719984                  0.493710   
Kenya     Holographic Goose            5.470392                       NaN   
          Kaggle                      35.286273                  0.486329   
          Kaggle Tiers                29.040282                  0.485873   
          Kerneler                    15.646461                  0.484390   
          Kerneler Dark Mode          18.41298

### the reason existing NAN because there are missing value in the num_sold column